In [ ]:
clean_22 = pd.read_pickle(r'D:\capstone_F24\data\Cleaned_data\df_filled_22.pkl')
clean_23 = pd.read_pickle(r'D:\capstone_F24\data\Cleaned_data\df_filled_23.pkl')
clean_22.head()

In [ ]:
# Create a station info dataframe
df_2022 = pd.read_pickle(r'D:\capstone_F24\data\MTA_Subway_Hourly_Ridership_2022.pkl')
df_2023 = pd.read_pickle(r'D:\capstone_F24\data\MTA_Subway_Hourly_Ridership_2023.pkl')
df_2022.head()

In [ ]:
ridership_data_all_years = pd.concat([df_2022, df_2023], axis=0)

In [ ]:
# Drop unnessessary columns
columns_to_drop = ['transit_timestamp','transit_mode','payment_method','fare_class_category','ridership','transfers']
ridership_data_all_years.drop(columns_to_drop, axis=1, inplace=True)
ridership_data_all_years.head()

In [ ]:
# Remove stations that are not in the cleaned timeseries data

ridership_data_all_years[ ~ridership_data_all_years['station_complex_id'].isin(clean_22.index) ]
print(len(ridership_data_all_years['station_complex_id'].unique()))
print(len(clean_22.index))
ridership_data_all_years['station_complex_id'].unique()

In [ ]:
def get_attraction_names(ridership_data):
    """
    Add a column of nearby attraction names for each station.
    """
    # Create copy of data to avoid modifying original
    data = ridership_data.copy()
    
    # Initialize empty list for attraction names
    attraction_names = []
    
    for station in data['station_complex_id'].unique():
        # Get station's lat/long
        station_data = data[data['station_complex_id'] == station].iloc[0]
        lat_long = f"{station_data['latitude']}%2C{station_data['longitude']}"
        
        url = (f"https://api.content.tripadvisor.com/api/v1/location/nearby_search"
               f"?latLong={lat_long}&key=62E7CCFA411C4FF8BDDE0FB6BEE2C2F6&category=attractions&radius=.5&radiusUnit=mi&language=en")

        
        headers = {"accept": "application/json"}
        
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            
            # Get attractions data and extract names
            attractions = response.json().get('data', [])
            names = ', '.join([attr.get('name', '') for attr in attractions])
            
            attraction_names.append({
                'station_complex_id': station,
                'nearby_attractions': names
            })
            time.sleep(0.1)  # Small delay to avoid hitting rate limits
            
        except Exception as e:
            print(f"Error getting data for station {station}: {e}")
            attraction_names.append({
                'station_complex_id': station,
                'nearby_attractions': ''
            })
    
    # Convert to DataFrame and merge with original data
    attractions_df = pd.DataFrame(attraction_names)
    result = data.merge(attractions_df, on='station_complex_id', how='left')
    
    return result

In [ ]:
enhanced_data = get_attraction_names(ridership_data_all_years)
enhanced_data.head()

In [ ]:
# Convert to a csv file
enhanced_data.to_csv('station_data.csv', index=False)